In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from pandas.core.interchange.dataframe_protocol import DataFrame

In [2]:
def CalcRevenue(data):
    solutions  = list()
    
    prices = data.iloc[:,:4]
    production = data.iloc[:,4:]
    
    for i,row in prices.iterrows():
        
        hourlyRevMod = {}
        for j in range(production.shape[1]-1):
            
            hourlyRev = row['DA_PriceEUR'] * production.iloc[i,j+1]
            
            if production.iloc[i,j+1] > production.iloc[i,0]:
                hourlyRev = hourlyRev - row['BalancingPriceUpEUR'] * (production.iloc[i,j+1]-production.iloc[i,0])
            elif production.iloc[i,0] > production.iloc[i,j+1]:
                hourlyRev = hourlyRev + row['BalancingPriceDownEUR'] * (production.iloc[i,0]-production.iloc[i,j+1])
              
            hourlyRevMod[production.columns[j + 1]] = hourlyRev
           
        hourlyRevMod['HourDK'] = row['HourDK']
        solutions.append(hourlyRevMod)    
    
    solutions = pd.DataFrame(solutions)
    return solutions

In [3]:
def VisProduction(production):
    num_columns_to_plot = production.shape[1] - 1  # Exclude 'actProd' from the count
    fig, axes = plt.subplots(num_columns_to_plot, 1, figsize=(10, 5 * num_columns_to_plot))

    # Loop through each column after 'actProd'
    for j in range(1, production.shape[1]):  # Start from index 1 to skip 'actProd'
        label = str(production.columns[j])  # Get the column label
        color = (np.random.rand(), np.random.rand(), np.random.rand())
        axes[j-1].plot(production.index, production.iloc[:, j], label=label, marker='o', color=color)  # Plot forecast
        axes[j-1].plot(production.index, production['actProd'], label="Actual Production", marker='o', color='orange')  # Plot actual production
        axes[j-1].set_title(f'{label} vs Actual Production')  # Set title for the subplot
        axes[j-1].set_xlabel('Time')  # Set x-axis label
        axes[j-1].set_ylabel('Power Production')  # Set y-axis label
        axes[j-1].grid()  # Add a grid
        axes[j-1].legend()
    
    plt.title('Production vs. Prediction')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.legend()  # Show the legend
    plt.grid() 
    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [4]:
def VisPrices(prices):
    
    fig = plt.figure(figsize=(10, 5))
    for j in range(0, prices.shape[1]):
        label = str(prices.columns[j])  # Get the column label
        color = (np.random.rand(), np.random.rand(), np.random.rand())
        plt.plot(prices.index, prices.iloc[:, j], label=label, marker='o', color=color)
    
    plt.title('Price development')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()  
    plt.grid() 
    plt.show()

In [5]:
def VisRevenue(solutions):
    
    fig, ax = plt.subplots(figsize=(12, 6))

    # Die Positionen für die Balken festlegen
    bar_width = 0.25  # Breite der Balken
    x = np.arange(len(solutions))  # x-Positionen der Balken

    # Balken für jede Datenreihe zeichnen
    for i, column in enumerate(solutions.columns):
        ax.bar(x + (i - 1) * bar_width, solutions[column], width=bar_width, label=column)

    # Achsen und Titel einstellen
    ax.set_xlabel('Time')
    ax.set_ylabel('Revenue')
    ax.set_title('Revenues based on different ML Models')
    ax.set_xticks(x)
    ax.set_xticklabels(solutions.index.strftime('%Y-%m-%d %H:%M:%S'), rotation=45, ha='right')  # Zeitstempel formatieren
    ax.legend()  # Legende hinzufügen

    # Layout anpassen und Diagramm anzeigen
    plt.tight_layout()
    plt.show()

In [6]:
def Visualise(solutions,data):
    
    prices = data.iloc[:,:4]
    production = data.iloc[:,4:]
    production['HourDK'] = prices['HourDK']
    
    prices.set_index('HourDK', inplace=True)
    production.set_index('HourDK', inplace=True)
    solutions.set_index('HourDK', inplace=True)
    
    VisProduction(production)
    VisPrices(prices)
    
    VisRevenue(solutions)

In [45]:
# p_t_hat as decision variable
def OptimizationProblemEnergybid(prices_df):
    model = gp.Model()
    solutions  = list()
    capacity = 30000.0
    
    for index, row in prices_df.iterrows():
        
        p_t_hat = model.addVar(name="p_t_hat", lb=0, ub=capacity, vtype=GRB.INTEGER)
        z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
        z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)
        
        condition_down = model.addVar(name="condition_down", vtype=GRB.BINARY)
        condition_up = model.addVar(name="condition_up", vtype=GRB.BINARY)
        
        model.addConstr(z_up <= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_up >= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_down <= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(z_down >= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(condition_up + condition_down == 1)
        
    
        #defining the constraints
        model.addConstr(-p_t_hat <= 0)
        model.addConstr(p_t_hat <= capacity)
        model.addConstr(-z_down <= 0)
        model.addConstr(-z_up <= 0)
        
        model.setObjective((row['DA_PriceEUR']*p_t_hat)+(row['BalancingPriceDownEUR']*z_down-row['BalancingPriceUpEUR']*z_up), GRB.MAXIMIZE)
        
        model.optimize()
    
        if model.status == GRB.OPTIMAL:
            print("Optimal solution found!")
            print(f"Objective Value: {model.ObjVal}")
            
            values = {"Day-Ahead-Price": row['DA_PriceEUR'],
                      "BalancingPriceUp":row['BalancingPriceUpEUR'],
                      "BalancingPriceDown":row['BalancingPriceDownEUR'],
                      "p_t":row['p_t'],
                      "Revenue":model.ObjVal
                      }
            values.update({v.varName: v.x for v in model.getVars()})
        
            
            solutions.append({row['HourDK']:values})
                
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            model.computeIIS()
            model.write("infeasible.ilp")  # Write IIS to a file for review
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f"Infeasible constraint: {c.constrName}")
            
            
        elif model.status == GRB.UNBOUNDED:
            print("Model is unbounded.")
        else:
            print("Model status:", model.status)
            
        model.remove(model.getConstrs())
        model.remove(model.getVars())
        
    solutions = pd.DataFrame([v for d in solutions for v in d.values()],
                  index=[ts for d in solutions for ts in d.keys()])    
    
    return solutions

In [33]:
#Helping Method to limit the  
def ObservationPeriod(Start_Observation, End_Observation, data):
    Start_Observation = pd.to_datetime(Start_Observation)
    End_Observation = pd.to_datetime(End_Observation)
    condition = (data["HourDK"]>= Start_Observation) & (data["HourDK"]< End_Observation)
    data  = data[condition]
    return data
        

In [50]:
prices_df = pd.read_csv('./Data assignment 1/prices_merged_df_output.csv', delimiter=',')
features_df = pd.read_csv('./Data assignment 1/Feature data.csv', delimiter=',')
features_df['AKI Kalby Active Power'] = -1 * features_df['AKI Kalby Active Power']
#predicted_data = pd.read_csv("./Data assignment 1/power_prod_ReguReg.csv", delimiter=',')
data = prices_df.copy()
data['actProd'] = features_df['AKI Kalby Active Power']
#data["ReguReg"] = -1 * predicted_data['ReguReg']
data["HourDK"] = pd.to_datetime(data["HourDK"])

#Hier irgendwo muss ich die eigentlichen Zeitreihen hinklatschen
data = ObservationPeriod("2022-06-01 00:00:00", "2022-06-01 10:00:00", data)

data.reset_index(drop=True, inplace=True)
data.rename(columns={'actProd': 'p_t'}, inplace=True)

## P_t_Hat computation

In [52]:
# Computing the optimal biding strategy values based on the historic data
# Hourly Optimization Problem
data.rename(columns={'actProd': 'p_t'}, inplace=True)
solutions = OptimizationProblemEnergybid(data)

df_target = pd.DataFrame({
    'p_t_hat': solutions['p_t_hat']
})
data.rename(columns={'p_t': 'actProd'}, inplace=True)

#df_target.to_csv("./Data assignment 1/TargetValues_Model2.csv", index=False)
#data.to_csv("./Data assignment 1/Features_Model2.csv", index=False)
    

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x68efa9f5
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+03]
  Objective range  [2e-01, 3e-01]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 595.0154384
Found heuristic solution: objective 595.0279917

Root relaxation: objective 6.310181e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes  

## Revenue Calculation

In [28]:
def RevOptiCalc(data):
    
   
    
    

    
    
    

### Data Preperation

In [ ]:
#Revenue calculation based on the different production predictions
prices_df = pd.read_csv('./Data assignment 1/prices_merged_df_output.csv', delimiter=',')
production_df = pd.read_csv('./Data assignment 1/Final_power.csv', delimiter=',')

for column in production_df.columns:
    prices_df[column] = production_df[column] *-1

data = prices_df.copy()

data["HourDK"] = pd.to_datetime(data["HourDK"])
data = ObservationPeriod("2022-10-01 00:00:00", "2022-10-01 10:00:00", data)
data.reset_index(drop=True, inplace=True)



### Linear Regression

In [58]:
data.rename(columns={'Linear Regression': 'p_t'}, inplace=True)
LinRegRev = OptimizationProblemEnergybid(data)
data.rename(columns={'p_t': 'Linear Regression'}, inplace=True)

LinRegRev



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0xc806df77
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 8e+02]
  Objective range  [4e-02, 6e-01]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 32.0853549
Found heuristic solution: objective 32.1321149

Root relaxation: objective 5.070433e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    

,Day-Ahead-Price,BalancingPriceUp,BalancingPriceDown,p_t,Revenue,p_t_hat,z_down,z_up,condition_down,condition_up
2022-10-01 00:00:00,0.06359,0.63587,0.04021,797.363235,50.695836,797.0,0.363235,0.000000,1.0,0.0
2022-10-01 01:00:00,0.05000,0.63587,0.06359,734.125248,46.683025,-0.0,734.125248,0.000000,1.0,-0.0
2022-10-01 02:00:00,0.02148,0.05000,0.01907,620.836192,13.333546,620.0,0.836192,0.000000,1.0,-0.0
2022-10-01 03:00:00,0.01691,0.02148,0.01450,666.214162,11.265165,666.0,0.214162,0.000000,1.0,0.0
2022-10-01 04:00:00,0.01503,0.01691,0.01450,711.979560,10.701014,712.0,0.000000,0.020440,0.0,1.0
2022-10-01 05:00:00,0.01502,0.01503,0.01503,1118.794301,16.815478,0.0,1118.794301,0.000000,1.0,-0.0
2022-10-01 06:00:00,0.02384,0.01502,0.01502,1746.006137,290.825012,30000.0,0.000000,28253.993863,0.0,1.0
2022-10-01 07:00:00,0.04502,0.02384,0.01700,2000.156975,683.083742,30000.0,0.000000,27999.843025,0.0,1.0
2022-10-01 08:00:00,0.07305,0.04502,0.02689,2764.532204,965.359240,30000.0,0.000000,27235.467796,0.0,1.0
2022-10-01 09:00:00,0.10985,0.07305,0.04021,2834.183929,1311.037055,30000.0,0.000000,27165.816071,0.0,1.0


### Non - Linear Regression 1

In [55]:
data.rename(columns={'Non-Linear Regression 1': 'p_t'}, inplace=True)
NonLinReg1Rev = OptimizationProblemEnergybid(data)
data.rename(columns={'p_t': 'Non-Linear Regression 1'}, inplace=True)

NonLinReg1Rev

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x5c63c153
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 6e+02]
  Objective range  [4e-02, 6e-01]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 25.1450426
Found heuristic solution: objective 25.1918026

Root relaxation: objective 3.972859e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    

,Day-Ahead-Price,BalancingPriceUp,BalancingPriceDown,p_t,Revenue,p_t_hat,z_down,z_up,condition_down,condition_up
2022-10-01 00:00:00,0.06359,0.63587,0.04021,624.761581,39.710783,624.0,0.761581,0.000000,1.0,-0.0
2022-10-01 01:00:00,0.05000,0.63587,0.06359,603.489876,38.375921,-0.0,603.489876,0.000000,1.0,-0.0
2022-10-01 02:00:00,0.02148,0.05000,0.01907,537.285805,11.540210,537.0,0.285805,0.000000,1.0,0.0
2022-10-01 03:00:00,0.01691,0.02148,0.01450,567.898750,9.602705,568.0,0.000000,0.101250,0.0,1.0
2022-10-01 04:00:00,0.01503,0.01691,0.01450,608.024597,9.138597,608.0,0.024597,0.000000,1.0,0.0
2022-10-01 05:00:00,0.01502,0.01503,0.01503,995.621681,14.964194,-0.0,995.621681,0.000000,1.0,-0.0
2022-10-01 06:00:00,0.02384,0.01502,0.01502,1786.451340,291.432499,30000.0,0.000000,28213.548660,0.0,1.0
2022-10-01 07:00:00,0.04502,0.02384,0.01700,2233.743668,688.652449,30000.0,0.000000,27766.256332,0.0,1.0
2022-10-01 08:00:00,0.07305,0.04502,0.02689,3376.049995,992.889771,30000.0,0.000000,26623.950005,0.0,1.0
2022-10-01 09:00:00,0.10985,0.07305,0.04021,3584.003448,1365.811373,30000.0,0.000000,26415.996552,0.0,1.0


### Non - Linear Regression 2

In [57]:
data.rename(columns={'Non-Linear Regression 2': 'p_t'}, inplace=True)
NonLinReg2Rev = OptimizationProblemEnergybid(data)
data.rename(columns={'p_t': 'Non-Linear Regression 2'}, inplace=True)

NonLinReg2Rev

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x8863b7fb
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 8e+02]
  Objective range  [4e-02, 6e-01]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 31.1035506
Found heuristic solution: objective 31.1503106

Root relaxation: objective 4.915166e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    

,Day-Ahead-Price,BalancingPriceUp,BalancingPriceDown,p_t,Revenue,p_t_hat,z_down,z_up,condition_down,condition_up
2022-10-01 00:00:00,0.06359,0.63587,0.04021,772.946315,49.129531,772.0,0.946315,0.000000,1.0,-0.0
2022-10-01 01:00:00,0.05000,0.63587,0.06359,700.104679,44.519657,-0.0,700.104679,0.000000,1.0,-0.0
2022-10-01 02:00:00,0.02148,0.05000,0.01907,572.041331,12.287348,572.0,0.041331,0.000000,1.0,0.0
2022-10-01 03:00:00,0.01691,0.02148,0.01450,663.816582,11.224300,664.0,0.000000,0.183418,0.0,1.0
2022-10-01 04:00:00,0.01503,0.01691,0.01450,725.370513,10.902122,725.0,0.370513,0.000000,1.0,0.0
2022-10-01 05:00:00,0.01502,0.01503,0.01503,1143.713189,17.190009,0.0,1143.713189,0.000000,1.0,-0.0
2022-10-01 06:00:00,0.02384,0.01502,0.01502,1635.753507,289.169018,30000.0,0.000000,28364.246493,0.0,1.0
2022-10-01 07:00:00,0.04502,0.02384,0.01700,1771.570095,677.634231,30000.0,0.000000,28228.429905,0.0,1.0
2022-10-01 08:00:00,0.07305,0.04502,0.02689,2500.378823,953.467055,30000.0,0.000000,27499.621177,0.0,1.0
2022-10-01 09:00:00,0.10985,0.07305,0.04021,2551.902163,1290.416371,30000.0,0.000000,27448.097837,0.0,1.0
